In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [0]:
## 피마인디언 당뇨병 예측

# 보통 당뇨병의 원인은 식습관 유전으로 꼽음
# 고립된 지역인 피마 지역의 인디언들은 고유의 혈통이
# 지속되어 왔지만, 20세기 후반 강제 이주 정책으로 인해
# 서구화된 식습관으로 많은 당뇨병 환자가 생겨남

# diabetes 데이터 셋 구조
# 임신횟수, 포도당부하검사, 혈압, 삼두근피하지방측정값, 
# 인슐린농도, 비만지수, 당뇨내력가중치, 나이, 당뇨여부


# 로지스틱회귀를 이용해서 당뇨병 예측에 대한 분석을 실시함
# 정확도, 정밀도, 재현율, F1 score, ROC, AUC

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

# pandas 출력 설정 : 출력시 가로 생략 없애기
# pd.set_option('display.max_columns', 50)
# pd.set_option('display.width',250)
pd.set_option('display.expand_frame_repr', False)

# 데이터 읽어오기
pima = pd.read_csv('diabetes2.csv') 
pima.info()

# 레이블 분포 확인
pima['Outcome'].value_counts()

# null값 확인
pima.info() # null값 없음
pima.head()
pima.tail()

# 데이터 train/test로 분리
pima_target = pima['Outcome']
pima_data = pima.drop(['Outcome'], axis=1)

X_train, X_test, y_train, y_test = \
    train_test_split(pima_data, pima_target, test_size=0.2,
                     stratify=pima_target, random_state=1907021030)
                    # stratify : 당뇨병 여부 분포 비율에 맞춰
                    # 학습/평가 데이터를 일정하게 나눔


# 로지스틱 회귀로 학습시킨 후 평가함
lr_clf = LogisticRegression()
lr_clf.fit(X_train, y_train)
pred = lr_clf.predict(X_test)

# 결과확인 1
print('오차행렬', confusion_matrix(y_test, pred))
print('정확도', accuracy_score(y_test, pred))
print('정밀도', precision_score(y_test, pred))
print('민감도(재현율)', recall_score(y_test, pred))
print('F1 스코어', f1_score(y_test, pred))



In [0]:
# 결과확인 2

# 각 예측확률별 특이도와 민감도, 임계값을 구함
# fprs, tprs, thresholds = roc_curve(y_test, pred)
# # ROC 그래프 그림
# plt.plot(fprs, tprs, 'r-', label='ROC')
# plt.plot([0,1], [0,1], 'k--', label='boundary')
# plt.show()

pred_proba = lr_clf.predict_proba(X_test)[:,1]
# 각 예측확률별 특이도와 민감도, 임계값을 구함
fprs, tprs, thresholds = roc_curve(y_test, pred_proba)
# ROC 그래프 그림
plt.plot(fprs, tprs, 'r-', label='ROC')
plt.plot([0,1], [0,1], 'k--', label='boundary')
plt.show()


print('AUC 스코어', roc_auc_score(y_test, pred))
# 70~80 : normal
# => 지표가 좋게 나온것은 아님
# => 데이터를 다시 살펴서 전처리할 내용여부 확인
# => 다른 알고리즘을 적용해 봄

# pima 데이터 다시 확인
pima.describe()

# 정제되지 않은 순수 데이터에서는 누락치가 많이 존재
# 혈당, 혈압, 지방측정값, 인슐린, 비만지수 등이 0 임


# 히스토그램으로 0비율이 어느정도인지 확인
plt.hist(pima_data['Glucose'])
plt.show()

plt.hist(pima_data['SkinThickness'])
plt.show()

plt.hist(pima_data['Insulin'])
plt.show()



# 각 변수별 0값의 분포 및 비율 확인
# 이것을 통해 0을 대체값으로 채울지 
# 아니면 과감하게 제거할 것인지 결정
pima_data[pima_data['Glucose'] == 0].count()
# Glucose 변수의 값이 0인 행의 수를 출력
# Glucose 과 관련없는 다른 변수들까지 출력됨 

pima_data[pima_data['Glucose'] == 0]['Glucose'].count()
# Glucose 변수의 값이 0인 행의 수를 출력


check_features = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

total = pima_data['BMI'].shape[0]   # 비율계산을 위해 전체 갯수 계산

for feature in check_features:
  zerocount = pima_data[pima_data[feature] == 0]['Glucose'].count()
  print('{0}: 총 건수 {1}, 비율 {2:.2f}%'.format(
      feature, zerocount, 100 * zerocount/total))

  
# 지방 측정값과 인슐린의 0값이 비율이 제법 크므로 
# 이들의 평균값을 대체값으로 채워 넣음
pima_data[check_features] = \
pima_data[check_features].\
replace(0, pima_data[check_features].mean())


# 대체값으로 채운 결과 다시 확인
pima_data.describe()
# => 각 변수의 값의 범위가 10단, 100단위로 다름
# 따라서, 변수값의 범위를 일정하게 만드는 정규화가 필요

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

pima_data_scaled = scaler.fit_transform(pima_data)

# 정규화 시킨 결과 확인
pd.DataFrame(pima_data_scaled).describe()

# 로지스틱 회귀 분석 다시 실시
X_train, X_test, y_train, y_test = \
train_test_split(pima_data_scaled, pima_target, test_size=0.2, \
                 stratify=pima_target, random_state=1907021150)





# 결과확인1






# 결과확인1








In [38]:
# scikit-learn 에서 제동하는 피마인디언 당뇨병 데이터

from sklearn.datasets import load_diabetes


diabetes = load_diabetes()

df = pd.DataFrame(diabetes.data, columns = diabetes.feature_names)

df.describe()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
count,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02
mean,-3.634285e-16,1.308343e-16,-8.045349e-16,1.281655e-16,-8.835316e-17,1.327024e-16,-4.574646e-16,3.777301e-16,-3.830854e-16,-3.412882e-16
std,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02
min,-1.072256e-01,-4.464164e-02,-9.027530e-02,-1.123996e-01,-1.267807e-01,-1.156131e-01,-1.023071e-01,-7.639450e-02,-1.260974e-01,-1.377672e-01
25%,-3.729927e-02,-4.464164e-02,-3.422907e-02,-3.665645e-02,-3.424784e-02,-3.035840e-02,-3.511716e-02,-3.949338e-02,-3.324879e-02,-3.317903e-02
50%,5.383060e-03,-4.464164e-02,-7.283766e-03,-5.670611e-03,-4.320866e-03,-3.819065e-03,-6.584468e-03,-2.592262e-03,-1.947634e-03,-1.077698e-03
75%,3.807591e-02,5.068012e-02,3.124802e-02,3.564384e-02,2.835801e-02,2.984439e-02,2.931150e-02,3.430886e-02,3.243323e-02,2.791705e-02
max,1.107267e-01,5.068012e-02,1.705552e-01,1.320442e-01,1.539137e-01,1.987880e-01,1.811791e-01,1.852344e-01,1.335990e-01,1.356118e-01
